In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


In [ ]:
import env 
import acquire
import prepare


# ACQUIRE

In [ ]:
raw_df = acquire.get_telco_chunk()

In [ ]:
raw_df.info()

### It's acquired, now
# PREPARE

In [ ]:
df = prepare.prep_telco()

### MAKE THE Test/Train, X/Y

In [ ]:
X = df.drop('churn', axis=1)
#X = df[['monthly_charges']]

In [ ]:
y = df['churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=123)

In [ ]:
X_train.head()

### PREP is boring, Let's 
# MODEL AND CHART STUFF. 
# OR EXPLORE. 
I guess.

In [ ]:
sns.countplot(x='churn', data=df)

In [ ]:
sns.countplot(x='churn', data=df, hue='dependents')

In [ ]:
#Boxplot to show the distribution of ages. X is class, Y is age.
plt.figure(figsize = (10,7))
sns.boxplot(x='churn', y='monthly_charges', data=df)

# MODEL, log-reg

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

In [ ]:
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_train)
predictions_probas = logmodel.predict_proba(X_train)

In [ ]:
predictions_probas[:,0] < .5

In [ ]:
from sklearn import metrics

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_train, predictions)

In [ ]:
cnf_matrix

In [ ]:
#Looks like using logistic regression with just monthly charges as the predictor is worthless and predicts nobody
#will churn